In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import kagglehub

path = kagglehub.dataset_download("phantasm34/emovdb-sorted")
print("Downloaded to:", path)

/data/b22_shruti_chaudhary/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Downloaded to: /data/b22_shruti_chaudhary/.cache/kagglehub/datasets/phantasm34/emovdb-sorted/versions/1


In [4]:
import os
import soundfile as sf
import pandas as pd
from tqdm import tqdm

DATA_DIR = path  # path returned by kagglehub

rows = []

for speaker in sorted(os.listdir(DATA_DIR)):
    spk_path = os.path.join(DATA_DIR, speaker)
    if not os.path.isdir(spk_path):
        continue
    
    for root, dirs, files in os.walk(spk_path):
        for f in files:
            if not f.lower().endswith((".wav", ".flac", ".mp3")):
                continue
            
            filepath = os.path.join(root, f)
            emotion = os.path.basename(os.path.dirname(filepath))
            
            try:
                info = sf.info(filepath)
                sr = info.samplerate
                duration = info.frames / info.samplerate
                channels = info.channels
            except:
                sr = None
                duration = None
                channels = None
            
            rows.append({
                "speaker": speaker,
                "emotion": emotion,
                "filepath": filepath,
                "sample_rate": sr,
                "duration_s": duration,
                "channels": channels,
            })

df = pd.DataFrame(rows)
df.to_csv("emovdb_manifest.csv", index=False)

df.head(), df.shape


(  speaker  emotion                                           filepath  \
 0     bea  Neutral  /data/b22_shruti_chaudhary/.cache/kagglehub/da...   
 1     bea  Neutral  /data/b22_shruti_chaudhary/.cache/kagglehub/da...   
 2     bea  Neutral  /data/b22_shruti_chaudhary/.cache/kagglehub/da...   
 3     bea  Neutral  /data/b22_shruti_chaudhary/.cache/kagglehub/da...   
 4     bea  Neutral  /data/b22_shruti_chaudhary/.cache/kagglehub/da...   
 
    sample_rate  duration_s  channels  
 0        44100    3.322630         1  
 1        44100    3.878390         1  
 2        44100    5.015170         1  
 3        44100    4.597846         1  
 4        44100    3.477188         1  ,
 (6893, 6))

In [5]:
df.tail()

,speaker,emotion,filepath,sample_rate,duration_s,channels
6888,sam,Angry,/data/b22_shruti_chaudhary/.cache/kagglehub/da...,16000,5.564938,1
6889,sam,Angry,/data/b22_shruti_chaudhary/.cache/kagglehub/da...,16000,4.385187,1
6890,sam,Angry,/data/b22_shruti_chaudhary/.cache/kagglehub/da...,16000,3.667500,1
6891,sam,Angry,/data/b22_shruti_chaudhary/.cache/kagglehub/da...,16000,4.655062,1
6892,sam,Angry,/data/b22_shruti_chaudhary/.cache/kagglehub/da...,16000,2.833500,1


In [15]:
! pip uninstall transformers -y

Found existing installation: transformers 5.0.0.dev0
Uninstalling transformers-5.0.0.dev0:
  Successfully uninstalled transformers-5.0.0.dev0


In [16]:
! pip install transformers torchaudio soundfile librosa tqdm


Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
Using cached huggingface_hub-0.36.0-py3-none-any.whl (566 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.1.7
    Uninstalling huggingface_hub-1.1.7:
      Successfully uninstalled huggingface_hub-1.1.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]


In [7]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA RTX A6000


In [17]:
import os
import torch
import torchaudio
import soundfile as sf
import pandas as pd
from tqdm import tqdm
from transformers import HubertModel, Wav2Vec2FeatureExtractor

ImportError: cannot import name 'HubertModel' from 'transformers' (/data/b22_shruti_chaudhary/.local/lib/python3.10/site-packages/transformers/__init__.py)

In [19]:
# path to manifest file from previous step
manifest_path = "emovdb_manifest.csv"
df = pd.read_csv(manifest_path)

In [9]:
import os
import torch
import torchaudio
import soundfile as sf
import pandas as pd
from tqdm import tqdm
from transformers import HubertModel, Wav2Vec2FeatureExtractor

manifest_path = "emovdb_manifest.csv"
df = pd.read_csv(manifest_path)

# HuBERT does NOT use a tokenizer. Only feature extractor.
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(
    "facebook/hubert-base-ls960"
)

model = HubertModel.from_pretrained("facebook/hubert-base-ls960").to("cuda")
model.eval()

all_features = []
all_paths = []
all_lengths = []

def load_audio(filepath, target_sr=16000):
    wav, sr = sf.read(filepath)
    wav = torch.tensor(wav).float()
    if wav.ndim > 1:
        wav = wav.mean(dim=1)
    if sr != target_sr:
        wav = torchaudio.functional.resample(wav, sr, target_sr)
    return wav

for fp in tqdm(df["filepath"], desc="Extracting HuBERT features"):
    wav = load_audio(fp).to("cuda")

    with torch.no_grad():
        inp = feature_extractor(
            wav, sampling_rate=16000, return_tensors="pt", padding=True
        )
        inp = {k: v.to("cuda") for k, v in inp.items()}

        out = model(**inp, output_hidden_states=True)
        feats = out.hidden_states[9].squeeze(0).cpu()

    all_features.append(feats)
    all_paths.append(fp)
    all_lengths.append(feats.shape[0])

output = {
    "paths": all_paths,
    "features": all_features,
    "lengths": all_lengths,
    "sample_rate": 16000,
}

torch.save(output, "hubert_features.pt")
print("Saved → hubert_features.pt")


preprocessor_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]


Extracting HuBERT features: 100%|██████████| 6893/6893 [05:17<00:00, 21.74it/s]


Saved → hubert_features.pt


In [10]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 10.9 MB/s eta 0:00:00:00:0100:01


In [11]:
import torch
import faiss
from tqdm import tqdm

# load continuous HuBERT features
data = torch.load("hubert_features.pt")
all_feats = data["features"]     # list of tensors (L_i, 768)
all_paths = data["paths"]

# ====== 1. Stack features into one big matrix ======
print("Stacking all features...")
feat_list = [f for f in all_feats]
X = torch.cat(feat_list, dim=0).float()      # shape: (N_frames, 768)
print("Total frames:", X.shape)

# convert to numpy for faiss
X_np = X.numpy().astype('float32')

# ====== 2. Train KMeans with K=200 ======
K = 200
print("Training K-Means (K=200)...")

kmeans = faiss.Kmeans(
    d=768,       # feature dimension
    k=K,
    niter=30,
    verbose=True,
    gpu=False
)

kmeans.train(X_np)

centroids = kmeans.centroids

# ====== 3. Quantize each utterance ======
print("Quantizing sequences...")

def quantize_seq(tensor, kmeans):
    """
    Input: (L, 768) tensor
    Output: unit_ids (L,)
    """
    x = tensor.numpy().astype('float32')
    _, ids = kmeans.index.search(x, 1)  # nearest centroid
    return ids.squeeze(1).tolist()

# ====== 4. Dedup ======
def dedupe(seq):
    """Remove consecutive repeated units."""
    if len(seq) == 0:
        return seq
    new = [seq[0]]
    for i in range(1, len(seq)):
        if seq[i] != seq[i-1]:
            new.append(seq[i])
    return new

all_units = []
all_units_dedup = []

for feats in tqdm(all_feats, desc="Converting to discrete units"):
    unit_ids = quantize_seq(feats, kmeans)
    dedup_ids = dedupe(unit_ids)
    all_units.append(unit_ids)
    all_units_dedup.append(dedup_ids)

# ====== 5. Save everything ======
save_dict = {
    "paths": all_paths,
    "units": all_units,
    "units_dedup": all_units_dedup,
    "kmeans_centroids": centroids,
    "K": K
}

torch.save(save_dict, "discrete_units.pt")
print("Saved → discrete_units.pt")


Stacking all features...
Total frames: torch.Size([1703083, 768])
Training K-Means (K=200)...
Sampling a subset of 51200 / 1703083 for training
Clustering 51200 points in 768D to 200 clusters, redo 1 times, 30 iterations
  Preprocessing in 1.09 s
Quantizing sequences... search 3.04 s): objective=1.85115e+06 imbalance=1.115 nsplit=0       


Converting to discrete units: 100%|██████████| 6893/6893 [00:05<00:00, 1236.93it/s]


Saved → discrete_units.pt


In [ ]:
# Step: Speaker lookup table + emotion one-hot + dataset helper
import os
import json
import torch
import torch.nn as nn
import pandas as pd
from typing import List
from torch.utils.data import Dataset, DataLoader

# ----------------------------
# Config
# ----------------------------
DISCRETE_PT = "discrete_units.pt"       # from step 3
MANIFEST_CSV = "emovdb_manifest.csv"    # from earlier
SPEAKER_TABLE_FN = "speaker_table.pt"
LABEL_MAPPINGS_FN = "label_mappings.pt"
SPK_DIM = 128                           # embedding size for speaker lookup table
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ----------------------------
# Load data artifacts
# ----------------------------
discrete = torch.load(DISCRETE_PT, weights_only=False)

paths = discrete["paths"]               # list of filepaths in same order as units
units_dedup = discrete["units_dedup"]   # list of lists
# manifest for metadata (speaker/emotion inference)
df = pd.read_csv(MANIFEST_CSV)

# ensure paths are same ordering/containment: build a mapping from filepath -> (speaker,emotion)
meta_map = {}
for _, row in df.iterrows():
    path = os.path.abspath(row["filepath"])
    meta_map[path] = {"speaker": row["speaker"], "emotion": row["emotion"]}

# normalize the paths stored in discrete to absolute for matching
abs_paths = [os.path.abspath(p) for p in paths]

# ----------------------------
# Create speaker -> id mapping
# ----------------------------
# Collect speakers present in the manifest for the set of files we have
speakers = []
for p in abs_paths:
    if p in meta_map:
        spk = meta_map[p]["speaker"]
        speakers.append(spk)
    else:
        # fallback: infer speaker from path parts (last-2 folder)
        parts = p.split(os.sep)
        spk = parts[-3] if len(parts) >= 3 else "unknown"
        speakers.append(spk)

unique_speakers = sorted(list(set(speakers)))
speaker2id = {s: i for i, s in enumerate(unique_speakers)}
id2speaker = {i: s for s, i in speaker2id.items()}
num_speakers = len(unique_speakers)
print(f"Found {num_speakers} speakers: {unique_speakers}")

# ----------------------------
# Create emotion -> id mapping
# ----------------------------
# Collect emotions from manifest
emotions = []
for p in abs_paths:
    if p in meta_map:
        emo = meta_map[p]["emotion"]
    else:
        parts = p.split(os.sep)
        emo = parts[-2] if len(parts) >= 2 else "unknown"
    emotions.append(emo)

unique_emotions = sorted(list(set(emotions)))
emotion2id = {e: i for i, e in enumerate(unique_emotions)}
id2emotion = {i: e for e, i in emotion2id.items()}
num_emotions = len(unique_emotions)
print(f"Found {num_emotions} emotions: {unique_emotions}")

# ----------------------------
# Speaker lookup module (trainable embedding table)
# ----------------------------
class SpeakerLookup(nn.Module):
    def __init__(self, num_speakers: int, spk_dim: int):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_speakers, embedding_dim=spk_dim)
        # initialize embeddings (standard normal small init)
        nn.init.normal_(self.embedding.weight, mean=0.0, std=0.02)

    def forward(self, speaker_ids: torch.LongTensor):
        """
        speaker_ids: (B,) long tensor
        returns: (B, spk_dim) float tensor
        """
        return self.embedding(speaker_ids)

# instantiate
spk_module = SpeakerLookup(num_speakers=num_speakers, spk_dim=SPK_DIM).to(DEVICE)

# Save initial speaker table and mappings (weights will be updated during later training)
torch.save({
    "state_dict": spk_module.state_dict(),
    "speaker2id": speaker2id,
    "id2speaker": id2speaker,
    "spk_dim": SPK_DIM
}, SPEAKER_TABLE_FN)
print(f"Saved speaker table to {SPEAKER_TABLE_FN}")

# Save label mappings
torch.save({
    "emotion2id": emotion2id,
    "id2emotion": id2emotion,
    "num_emotions": num_emotions
}, LABEL_MAPPINGS_FN)
print(f"Saved label mappings to {LABEL_MAPPINGS_FN}")

# ----------------------------
# Utility functions
# ----------------------------
def get_speaker_id_from_path(path: str):
    p = os.path.abspath(path)
    if p in meta_map:
        spk = meta_map[p]["speaker"]
    else:
        parts = p.split(os.sep)
        spk = parts[-3] if len(parts) >= 3 else "unknown"
    return speaker2id.get(spk, -1)

def get_emotion_id_from_path(path: str):
    p = os.path.abspath(path)
    if p in meta_map:
        emo = meta_map[p]["emotion"]
    else:
        parts = p.split(os.sep)
        emo = parts[-2] if len(parts) >= 2 else "unknown"
    return emotion2id.get(emo, -1)

# one-hot helper
def emotion_one_hot(emo_id: int, num_classes: int):
    v = torch.zeros(num_classes, dtype=torch.float32)
    if emo_id >= 0:
        v[emo_id] = 1.0
    return v

# ----------------------------
# Dataset class that yields units + speaker + emotion conditioning
# ----------------------------
class UnitsDataset(Dataset):
    def __init__(self, paths: List[str], units_dedup: List[List[int]],
                 speaker2id: dict, emotion2id: dict, device: str = "cpu"):
        assert len(paths) == len(units_dedup)
        self.paths = [os.path.abspath(p) for p in paths]
        self.units = units_dedup
        self.speaker2id = speaker2id
        self.emotion2id = emotion2id
        self.device = device
        self.num_emotions = len(emotion2id)

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        p = self.paths[idx]
        unit_seq = torch.tensor(self.units[idx], dtype=torch.long)           # shape (L,)
        spk_id = get_speaker_id_from_path(p)
        emo_id = get_emotion_id_from_path(p)
        spk_id_t = torch.tensor(spk_id, dtype=torch.long)
        emo_onehot = emotion_one_hot(emo_id, self.num_emotions)
        return {
            "path": p,
            "units": unit_seq.to(self.device),
            "speaker_id": spk_id_t.to(self.device),
            "emotion_onehot": emo_onehot.to(self.device),
            "emotion_id": torch.tensor(emo_id, dtype=torch.long).to(self.device)
        }

# quick sanity: build dataset and sample
dataset = UnitsDataset(abs_paths, units_dedup, speaker2id, emotion2id, device=DEVICE)
print("Dataset size:", len(dataset))
sample = dataset[0]
print("Sample keys:", sample.keys())
print("Sample units length:", sample["units"].shape)
print("Sample speaker_id:", sample["speaker_id"].item())
print("Sample emotion_onehot sum (should be 1):", sample["emotion_onehot"].sum().item())

# optional: DataLoader example
dl = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=lambda batch: batch) 
batch = next(iter(dl))
print("Loaded batch of", len(batch), "items")

# ----------------------------
# Example: load speaker embedding module and get speaker vectors
# ----------------------------
loaded = torch.load(SPEAKER_TABLE_FN, map_location=DEVICE)
spk_mod = SpeakerLookup(num_speakers=len(loaded["speaker2id"]), spk_dim=loaded["spk_dim"]).to(DEVICE)
spk_mod.load_state_dict(loaded["state_dict"])
spk_mod.eval()

# get vector for a sample
sample_spk_id = dataset[0]["speaker_id"].unsqueeze(0)  # shape (1,)
with torch.no_grad():
    spk_vec = spk_mod(sample_spk_id.to(DEVICE))         # (1, SPK_DIM)
print("Speaker vector shape:", spk_vec.shape)


In [21]:
path="/data/b22_shruti_chaudhary/.cache/kagglehub/datasets/phantasm34/emovdb-sorted/versions/1"

In [23]:
for p in paths[:10]:
    print(p)


/kaggle/input/emovdb-sorted/bea/Disgusted/disgust_1-28_0014.wav
/kaggle/input/emovdb-sorted/bea/Disgusted/disgust_169-196_0173.wav
/kaggle/input/emovdb-sorted/bea/Disgusted/disgust_169-196_0174.wav
/kaggle/input/emovdb-sorted/bea/Disgusted/disgust_29-56_0045.wav
/kaggle/input/emovdb-sorted/bea/Disgusted/disgust_141-168_0152.wav
/kaggle/input/emovdb-sorted/bea/Disgusted/disgust_309-336_0336.wav
/kaggle/input/emovdb-sorted/bea/Disgusted/disgust_225-252_0236.wav
/kaggle/input/emovdb-sorted/bea/Disgusted/disgust_197-224_0224.wav
/kaggle/input/emovdb-sorted/bea/Disgusted/disgust_253-280_0254.wav
/kaggle/input/emovdb-sorted/bea/Disgusted/disgust_253-280_0266.wav


In [25]:
import os
import torch
import pandas as pd
from collections import defaultdict

discrete = torch.load("discrete_units.pt", weights_only=False)
units = discrete["units_dedup"]
paths = discrete["paths"]

df = pd.read_csv("emovdb_manifest.csv")
label_map = torch.load("label_mappings.pt")
emotion2id = label_map["emotion2id"]

def to_rel(p):
    return "/".join(p.split("/")[-3:])

df["relpath"] = df["filepath"].apply(to_rel)
rel_paths = [to_rel(p) for p in paths]

def extract_sentence_id(rel_path):
    fname = os.path.basename(rel_path)
    base = fname.split(".")[0]
    parts = base.split("_")
    return parts[-1]  # "0007"

grouped = defaultdict(lambda: defaultdict(dict))

for rel_path, unit_seq in zip(rel_paths, units):
    meta = df[df["relpath"] == rel_path]
    if meta.empty:
        continue
    speaker = meta.iloc[0]["speaker"]
    emotion = meta.iloc[0]["emotion"]
    sent_id = extract_sentence_id(rel_path)
    grouped[speaker][sent_id][emotion] = (rel_path, unit_seq)

pairs = []

for speaker, sent_dict in grouped.items():
    for sent_id, emo_dict in sent_dict.items():
        emotions = list(emo_dict.keys())
        if len(emotions) < 2:
            continue
        for emo_src in emotions:
            for emo_tgt in emotions:
                if emo_src == emo_tgt:
                    continue
                src_path, src_seq = emo_dict[emo_src]
                tgt_path, tgt_seq = emo_dict[emo_tgt]
                pairs.append((
                    src_seq,
                    emotion2id[emo_src],
                    tgt_seq,
                    emotion2id[emo_tgt],
                    src_path
                ))

print("Total parallel pairs:", len(pairs))
print("Example pair:", pairs[0] if pairs else None)


Total parallel pairs: 21730
Example pair: ([187, 186, 32, 50, 32, 193, 50, 31, 83, 193, 179, 42, 192, 129, 104, 44, 104, 87, 180, 29, 152, 91, 162, 148, 166, 57, 156, 2, 107, 109, 158, 35, 98, 33, 76, 22, 182, 139, 199, 146, 0, 121, 144, 63, 118, 68, 24, 9, 134, 2, 107, 63, 24, 123, 21, 165, 139, 25, 134, 45, 106, 71, 73, 153, 77, 4, 124, 156, 45, 6, 115, 153, 77, 133, 199, 193, 25, 102, 69, 134, 14, 93, 160, 170, 55, 163, 168, 34, 163, 140, 12, 184, 188, 189, 106, 182, 146, 174, 15, 77, 125, 146, 48, 142, 59, 175, 63, 66, 24, 9, 139, 78, 164, 49, 108, 13, 100, 195, 129, 154, 172], 2, [187, 186, 32, 193, 32, 88, 193, 31, 88, 50, 88, 152, 193, 152, 193, 152, 193, 148, 193, 148, 57, 8, 156, 2, 107, 130, 109, 92, 35, 98, 76, 22, 182, 139, 146, 48, 0, 10, 121, 144, 63, 185, 118, 68, 124, 24, 9, 78, 134, 2, 107, 63, 124, 9, 139, 78, 21, 165, 199, 25, 134, 45, 106, 71, 73, 153, 77, 133, 124, 156, 123, 45, 71, 73, 153, 77, 102, 69, 134, 14, 93, 160, 173, 188, 189, 106, 182, 146, 115, 153, 96,

In [26]:
# import os
# import torch
# import pandas as pd
# from collections import defaultdict
# import numpy as np
# from torch.serialization import add_safe_globals

# add_safe_globals([np.core.multiarray._reconstruct])

# discrete = torch.load("discrete_units.pt", weights_only=False)
# units = discrete["units_dedup"]
# paths = [os.path.abspath(p) for p in discrete["paths"]]

# df = pd.read_csv("emovdb_manifest.csv")
# label_map = torch.load("label_mappings.pt")
# emotion2id = label_map["emotion2id"]


# def extract_sentence_id(path):
#     fname = os.path.basename(path)
#     base = fname.split(".")[0]
#     parts = base.split("_")
#     return parts[-1]  # "0007"

# # Group: speaker -> sentence_id -> emotion -> unit_seq
# grouped = defaultdict(lambda: defaultdict(dict))

# for path, unit_seq in zip(paths, units):
#     meta = df[df["filepath"] == path]
#     if meta.empty:
#         continue
#     speaker = meta.iloc[0]["speaker"]
#     emotion = meta.iloc[0]["emotion"]
#     sent_id = extract_sentence_id(path)
#     grouped[speaker][sent_id][emotion] = (path, unit_seq)

# pairs = []

# # Build pairs only where >=2 emotions exist
# for speaker, sent_dict in grouped.items():
#     for sent_id, emo_dict in sent_dict.items():
#         emotions = list(emo_dict.keys())
#         if len(emotions) < 2:
#             continue
#         for emo_src in emotions:
#             for emo_tgt in emotions:
#                 if emo_src == emo_tgt:
#                     continue
#                 src_path, src_seq = emo_dict[emo_src]
#                 tgt_path, tgt_seq = emo_dict[emo_tgt]
#                 pairs.append((
#                     src_seq,
#                     emotion2id[emo_src],
#                     tgt_seq,
#                     emotion2id[emo_tgt],
#                     src_path
#                 ))

# print("Total parallel pairs:", len(pairs))
# print("Example pair:", pairs[0])


In [ ]:
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [28]:
import os
import math
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd

K = 200
PAD = 0
SHIFT = 1
BOS = K + SHIFT      # 201
EOS = K + SHIFT + 1  # 202
VOCAB_SIZE = K + SHIFT + 2  # 203

device = "cuda" if torch.cuda.is_available() else "cpu"

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.pe = pe.unsqueeze(0)
    def forward(self, x):
        return self.pe[:, :x.size(1)].to(x.device)

class EmotionTransformer(nn.Module):
    def __init__(self, num_emotions, vocab_size=VOCAB_SIZE, d_model=512, nhead=8, num_layers=6, dim_feedforward=2048, dropout=0.1, max_len=4096):
        super().__init__()
        self.num_emotions = num_emotions
        self.vocab_size = vocab_size
        self.token_emb = nn.Embedding(vocab_size, d_model, padding_idx=PAD)
        self.pos_enc = PositionalEncoding(d_model, max_len=max_len)
        self.encoders = nn.ModuleDict()
        self.decoders = nn.ModuleDict()
        enc_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        dec_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        for e in range(num_emotions):
            self.encoders[str(e)] = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
            self.decoders[str(e)] = nn.TransformerDecoder(dec_layer, num_layers=num_layers)
        self.output_proj = nn.Linear(d_model, vocab_size)
        self.d_model = d_model
    def encode(self, src_tokens, src_mask, emotion_id):
        e = str(int(emotion_id))
        x = self.token_emb(src_tokens) * math.sqrt(self.d_model)
        x = x + self.pos_enc(x)
        x = x.transpose(0,1)
        memory = self.encoders[e](x, src_key_padding_mask=src_mask)
        return memory
    def decode(self, tgt_tokens, memory, memory_mask, emotion_id, tgt_mask):
        e = str(int(emotion_id))
        y = self.token_emb(tgt_tokens) * math.sqrt(self.d_model)
        y = y + self.pos_enc(y)
        y = y.transpose(0,1)
        out = self.decoders[e](y, memory, tgt_mask=tgt_mask, memory_key_padding_mask=memory_mask)
        out = out.transpose(0,1)
        logits = self.output_proj(out)
        return logits
    def forward(self, src_tokens, src_mask, tgt_tokens, tgt_mask, src_em, tgt_em):
        memory = self.encode(src_tokens, src_mask, src_em)
        if tgt_mask is None:
            tgt_mask = self.generate_square_subsequent_mask(tgt_tokens.size(1)).to(tgt_tokens.device)
        logits = self.decode(tgt_tokens, memory, src_mask, tgt_em, tgt_mask)
        return logits
    def generate_square_subsequent_mask(self, sz):
        return torch.triu(torch.full((sz, sz), float('-inf')), diagonal=1)

class PairedUnitsDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs
    def __len__(self):
        return len(self.pairs)
    def __getitem__(self, idx):
        a, a_emo, b, b_emo, path_a = self.pairs[idx]
        return {
            "src_units": torch.tensor([x+SHIFT for x in a], dtype=torch.long),
            "src_emotion": a_emo,
            "tgt_units": torch.tensor([x+SHIFT for x in b], dtype=torch.long),
            "tgt_emotion": b_emo,
            "path": path_a
        }

def collate_pairs(batch):
    srcs = [b["src_units"].tolist() for b in batch]
    tgts = [b["tgt_units"].tolist() for b in batch]
    src_em = torch.tensor(batch[0]["src_emotion"], dtype=torch.long)
    tgt_em = torch.tensor(batch[0]["tgt_emotion"], dtype=torch.long)
    max_src = max(len(s) for s in srcs)
    max_tgt = max(len(t) for t in tgts) + 1
    src_padded, src_mask, tgt_in, tgt_out = [], [], [], []
    for s, t in zip(srcs, tgts):
        sseq = s + [PAD]*(max_src-len(s))
        smask = [False]*len(s) + [True]*(max_src-len(s))
        tgt_seq = [BOS] + t + [EOS]
        tgt_in_seq = tgt_seq[:-1] + [PAD]*(max_tgt-(len(tgt_seq)-1))
        tgt_out_seq = tgt_seq[1:] + [PAD]*(max_tgt-(len(tgt_seq)-1))
        src_padded.append(sseq)
        src_mask.append(smask)
        tgt_in.append(tgt_in_seq)
        tgt_out.append(tgt_out_seq)
    return {
        "src": torch.tensor(src_padded, dtype=torch.long).to(device),
        "src_mask": torch.tensor(src_mask, dtype=torch.bool).to(device),
        "tgt_in": torch.tensor(tgt_in, dtype=torch.long).to(device),
        "tgt_out": torch.tensor(tgt_out, dtype=torch.long).to(device),
        "src_em": src_em.to(device),
        "tgt_em": tgt_em.to(device),
    }

def make_denoising_input(units, p_mask=0.15, span_len=3):
    seq = units.copy()
    L = len(seq)
    i = 0
    while i < L:
        if random.random() < p_mask:
            span = random.randint(1, span_len)
            end = min(i+span, L)
            for j in range(i, end):
                seq[j] = K - 1  # use max raw unit as noise
            i = end
        else:
            i += 1
    return seq

def pretrain_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    for batch in dataloader:
        items = []
        for i in range(len(batch)):
            units = batch[i]["units"].tolist()      # raw 0..K-1
            emo = batch[i]["emotion_id"]
            corrupted = make_denoising_input(units)
            items.append((corrupted, emo, units, emo, ""))
        paired = PairedUnitsDataset(items)
        pl = DataLoader(paired, batch_size=len(items), shuffle=False, collate_fn=collate_pairs)
        for pb in pl:
            optimizer.zero_grad()
            logits = model(pb["src"], pb["src_mask"], pb["tgt_in"], None, pb["src_em"], pb["tgt_em"])
            loss = criterion(logits.view(-1, model.vocab_size), pb["tgt_out"].view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
    return total_loss

def finetune_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    for batch in dataloader:
        optimizer.zero_grad()
        logits = model(batch["src"], batch["src_mask"], batch["tgt_in"], None, batch["src_em"], batch["tgt_em"])
        loss = criterion(logits.view(-1, model.vocab_size), batch["tgt_out"].view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss

def greedy_decode(model, src, src_mask, src_em, tgt_em, max_len=300):
    model.eval()
    with torch.no_grad():
        memory = model.encode(src, src_mask, src_em)
        ys = torch.tensor([[BOS]], dtype=torch.long).to(device)
        for _ in range(max_len):
            tgt_mask = model.generate_square_subsequent_mask(ys.size(1)).to(device)
            out = model.decode(ys, memory, src_mask, tgt_em, tgt_mask)
            nxt = out[:, -1, :].argmax(-1, keepdim=True)
            ys = torch.cat([ys, nxt], dim=1)
            if nxt.item() == EOS:
                break
    return ys.squeeze(0).tolist()

discrete = torch.load("discrete_units.pt", weights_only=False)
paths = discrete["paths"]
units_dedup = discrete["units_dedup"]

df = pd.read_csv("emovdb_manifest.csv")
manifest = {os.path.abspath(r["filepath"]): {"speaker": r["speaker"], "emotion": r["emotion"]} for _, r in df.iterrows()}
emotion2id = torch.load("label_mappings.pt")["emotion2id"]

num_emotions = len(emotion2id)
model = EmotionTransformer(num_emotions=num_emotions).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=PAD)

emo_ids = []
abs_paths = [os.path.abspath(p) for p in paths]
for p in abs_paths:
    emo = manifest.get(p, {}).get("emotion", None)
    emo_ids.append(emotion2id.get(emo, 0))

class SimpleUnitsListDataset(Dataset):
    def __init__(self, paths, units, emo_ids):
        self.paths = paths
        self.units = units
        self.emo_ids = emo_ids
    def __len__(self):
        return len(self.paths)
    def __getitem__(self, idx):
        return {"path": self.paths[idx], "units": torch.tensor(self.units[idx], dtype=torch.long), "emotion_id": self.emo_ids[idx]}

pretrain_ds = SimpleUnitsListDataset(abs_paths, units_dedup, emo_ids)
pretrain_loader = DataLoader(pretrain_ds, batch_size=8, shuffle=True, collate_fn=lambda b: b)


num_pretrain_epochs = 15

for epoch in range(1, num_pretrain_epochs + 1):
    running_loss = 0.0
    batches = 0

    # display epoch header
    print(f"\n--- Pretrain Epoch {epoch}/{num_pretrain_epochs} ---")

    # tqdm batch loop
    for batch in tqdm(pretrain_loader, desc=f"Pretrain {epoch}", leave=False):
        loss = pretrain_epoch(model, [batch], optimizer, criterion)  # wrap batch to mimic loader
        running_loss += loss
        batches += 1

    avg_loss = running_loss / batches
    print(f"pretrain epoch {epoch} loss: {avg_loss:.4f}")

# for epoch in range(15):
#     loss = pretrain_epoch(model, pretrain_loader, optimizer, criterion)
#     print("pretrain loss", loss)

# 'pairs' must be defined before this (built from your parallel pairing step)
paired_ds = PairedUnitsDataset(pairs)
paired_loader = DataLoader(paired_ds, batch_size=16, shuffle=False, collate_fn=collate_pairs)

num_finetune_epochs = 25

for epoch in range(1, num_finetune_epochs + 1):
    running_loss = 0.0
    batches = 0

    print(f"\n--- Finetune Epoch {epoch}/{num_finetune_epochs} ---")

    for batch in tqdm(paired_loader, desc=f"Finetune {epoch}", leave=False):
        loss = finetune_epoch(model, [batch], optimizer, criterion)
        running_loss += loss
        batches += 1

    avg_loss = running_loss / batches
    print(f"finetune epoch {epoch} loss: {avg_loss:.4f}")


# for epoch in range(25):
#     loss = finetune_epoch(model, paired_loader, optimizer, criterion)
#     print("finetune loss", loss)

torch.save(model.state_dict(), "es2s_transformer.pt")


/data/b22_shruti_chaudhary/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(



--- Pretrain Epoch 1/15 ---


pretrain epoch 1 loss: 2.4944

--- Pretrain Epoch 2/15 ---


pretrain epoch 2 loss: 1.9746

--- Pretrain Epoch 3/15 ---


pretrain epoch 3 loss: 1.7478

--- Pretrain Epoch 4/15 ---


...

finetune epoch 21 loss: 0.8328

--- Finetune Epoch 22/25 ---


finetune epoch 22 loss: 0.7858

--- Finetune Epoch 23/25 ---


finetune epoch 23 loss: 0.7415

--- Finetune Epoch 24/25 ---


finetune epoch 24 loss: 0.6975

--- Finetune Epoch 25/25 ---


finetune epoch 25 loss: 0.6574


In [29]:
! pip install pyworld soundfile librosa


Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyworld: filename=pyworld-0.3.5-cp310-cp310-linux_x86_64.whl size=887034 sha256=288804abed0d4ceb9ab53eab83d2dee8a16c90cb5033c016495c88f89b257647
  Stored in directory: /data/b22_shruti_chaudhary/.cache/pip/wheels/8e/a0/94/52e99161f9460670f11129bff5224ddf1a17915007d8cfa196
Successfully built pyworld


In [30]:
DATA_ROOT = "/data/b22_shruti_chaudhary/.cache/kagglehub/datasets/phantasm34/emovdb-sorted/versions/1"


In [31]:
import os
import torch
import soundfile as sf
import numpy as np
import pyworld as pw
import pandas as pd
from collections import defaultdict

DATA_ROOT = "/data/b22_shruti_chaudhary/.cache/kagglehub/datasets/phantasm34/emovdb-sorted/versions/1"
device = "cuda" if torch.cuda.is_available() else "cpu"

discrete = torch.load("discrete_units.pt", weights_only=False)
paths_kaggle = discrete["paths"]
units_full = discrete["units"]           # frame-level units (with repeats)
units_dedup = discrete["units_dedup"]    # dedup units

df = pd.read_csv("emovdb_manifest.csv")
label_map = torch.load("label_mappings.pt")
emotion2id = label_map["emotion2id"]

def to_rel(p):
    return "/".join(p.split("/")[-3:])

df["relpath"] = df["filepath"].apply(to_rel)
rel_from_kaggle = [to_rel(p) for p in paths_kaggle]

def get_local_path(rel):
    return os.path.join(DATA_ROOT, rel)

def extract_f0(x, sr=16000, frame_period_ms=20.0):
    f0, t = pw.harvest(x.astype(np.float64), sr, frame_period=frame_period_ms)
    f0 = pw.stonemask(x.astype(np.float64), f0, t, sr)
    return f0

def resample_f0_to_units(f0, target_len):
    src_idx = np.linspace(0, len(f0)-1, num=len(f0))
    tgt_idx = np.linspace(0, len(f0)-1, num=target_len)
    f0_interp = np.interp(tgt_idx, src_idx, f0)
    return f0_interp

def run_length_encode(seq):
    if len(seq) == 0:
        return [], []
    ids = []
    lens = []
    prev = seq[0]
    cnt = 1
    for x in seq[1:]:
        if x == prev:
            cnt += 1
        else:
            ids.append(prev)
            lens.append(cnt)
            prev = x
            cnt = 1
    ids.append(prev)
    lens.append(cnt)
    return ids, lens

prosody_entries = []

for rel, u_full, u_dedup in zip(rel_from_kaggle, units_full, units_dedup):
    meta = df[df["relpath"] == rel]
    if meta.empty:
        continue
    emo = meta.iloc[0]["emotion"]
    emo_id = emotion2id[emo]
    local_path = get_local_path(rel)
    if not os.path.exists(local_path):
        continue
    try:
        wav, sr = sf.read(local_path)
    except:
        continue
    if wav.ndim > 1:
        wav = wav.mean(axis=1)
    if sr != 16000:
        import librosa
        wav = librosa.resample(wav, orig_sr=sr, target_sr=16000)
        sr = 16000
    f0 = extract_f0(wav, sr=sr, frame_period_ms=20.0)
    u_full_np = np.array(u_full, dtype=np.int64)
    if len(f0) != len(u_full_np):
        f0 = resample_f0_to_units(f0, len(u_full_np))
    ids, lens = run_length_encode(u_full_np)
    if len(ids) != len(u_dedup):
        continue
    f0_per_unit = []
    idx = 0
    f0 = np.asarray(f0)
    for L in lens:
        seg = f0[idx:idx+L]
        if len(seg) == 0:
            f0_per_unit.append(0.0)
        else:
            f0_per_unit.append(float(seg.mean()))
        idx += L
    prosody_entries.append({
        "relpath": rel,
        "units_dedup": torch.tensor(u_dedup, dtype=torch.long),
        "f0": torch.tensor(f0_per_unit, dtype=torch.float32),
        "dur": torch.tensor(lens, dtype=torch.float32),
        "emotion_id": emo_id
    })

torch.save({"entries": prosody_entries}, "prosody_data.pt")
print("Saved prosody_data.pt with", len(prosody_entries), "utterances")


Saved prosody_data.pt with 6893 utterances


In [32]:
import torch
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader

prosody_data = torch.load("prosody_data.pt")
entries = prosody_data["entries"]

df = pd.read_csv("emovdb_manifest.csv")
df["relpath"] = df["filepath"].apply(lambda p: "/".join(p.split("/")[-3:]))

spk_table = torch.load("speaker_table.pt")
speaker2id = spk_table["speaker2id"]
spk_dim = spk_table["spk_dim"]

def get_speaker_id(relpath):
    row = df[df["relpath"] == relpath]
    if row.empty:
        return 0
    spk = row.iloc[0]["speaker"]
    return speaker2id.get(spk, 0)

class ProsodyDataset(Dataset):
    def __init__(self, entries):
        self.entries = entries
    def __len__(self):
        return len(self.entries)
    def __getitem__(self, idx):
        e = self.entries[idx]
        units = e["units_dedup"]
        f0 = e["f0"]
        dur = e["dur"]
        emo = e["emotion_id"]
        spk_id = get_speaker_id(e["relpath"])
        return {
            "units": units,
            "f0": f0,
            "dur": dur,
            "emo_id": torch.tensor(emo, dtype=torch.long),
            "spk_id": torch.tensor(spk_id, dtype=torch.long)
        }

def collate_prosody(batch):
    max_len = max(len(b["units"]) for b in batch)
    units, f0, dur, emo, spk, mask = [], [], [], [], [], []
    for b in batch:
        L = len(b["units"])
        pad_len = max_len - L
        units.append(torch.cat([b["units"], torch.zeros(pad_len, dtype=torch.long)]))
        f0.append(torch.cat([b["f0"], torch.zeros(pad_len)]))
        dur.append(torch.cat([b["dur"], torch.zeros(pad_len)]))
        emo.append(b["emo_id"])
        spk.append(b["spk_id"])
        mask.append(torch.cat([torch.zeros(L, dtype=torch.bool), torch.ones(pad_len, dtype=torch.bool)]))
    return {
        "units": torch.stack(units),
        "f0": torch.stack(f0),
        "dur": torch.stack(dur),
        "emo_id": torch.stack(emo),
        "spk_id": torch.stack(spk),
        "mask": torch.stack(mask)
    }

prosody_ds = ProsodyDataset(entries)
prosody_loader = DataLoader(prosody_ds, batch_size=16, shuffle=True, collate_fn=collate_prosody)
print(len(prosody_ds))


6893


In [34]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"

K = 200
PAD = 0
SHIFT = 1
VOCAB_SIZE = K + SHIFT + 2

class SpeakerLookup(nn.Module):
    def __init__(self, num_speakers, spk_dim):
        super().__init__()
        self.embedding = nn.Embedding(num_speakers, spk_dim)
    def forward(self, spk_ids):
        return self.embedding(spk_ids)

num_speakers = len(speaker2id)
spk_lookup = SpeakerLookup(num_speakers, spk_dim).to(device)
spk_lookup.load_state_dict(spk_table["state_dict"])

emotion2id = torch.load("label_mappings.pt")["emotion2id"]
num_emotions = len(emotion2id)

class ProsodyPredictor(nn.Module):
    def __init__(self, vocab_size, num_emotions, spk_dim, d_model=256, nhead=4, num_layers=3, ff=512):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, d_model, padding_idx=PAD)
        self.emo_emb = nn.Embedding(num_emotions, d_model)
        self.proj_spk = nn.Linear(spk_dim, d_model)
        enc_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=ff, batch_first=True)
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
        self.f0_head = nn.Linear(d_model, 1)
        self.dur_head = nn.Linear(d_model, 1)
    def forward(self, units, emo_ids, spk_vecs, src_key_padding_mask):
        x_tok = self.token_emb(units)
        x_emo = self.emo_emb(emo_ids).unsqueeze(1)
        x_spk = self.proj_spk(spk_vecs).unsqueeze(1)
        x = x_tok + x_emo + x_spk
        h = self.encoder(x, src_key_padding_mask=src_key_padding_mask)
        f0 = self.f0_head(h).squeeze(-1)
        dur = self.dur_head(h).squeeze(-1)
        return f0, dur

prosody_model = ProsodyPredictor(VOCAB_SIZE, num_emotions, spk_dim).to(device)
optim_p = torch.optim.Adam(prosody_model.parameters(), lr=1e-4)
mse = nn.MSELoss(reduction="none")

def train_prosody_epoch(model, loader, spk_lookup, optimizer):
    model.train()
    total_loss = 0.0
    for batch in loader:
        units = batch["units"].to(device)
        f0 = batch["f0"].to(device)
        dur = batch["dur"].to(device)
        emo = batch["emo_id"].to(device)
        spk_id = batch["spk_id"].to(device)
        mask = batch["mask"].to(device)
        with torch.no_grad():
            spk_vecs = spk_lookup(spk_id)
        optimizer.zero_grad()
        f0_pred, dur_pred = model(units, emo, spk_vecs, src_key_padding_mask=mask)
        f0_loss = mse(f0_pred, f0)
        dur_loss = mse(dur_pred, dur)
        f0_loss = (f0_loss * (~mask)).sum() / (~mask).sum().clamp(min=1)
        dur_loss = (dur_loss * (~mask)).sum() / (~mask).sum().clamp(min=1)
        loss = f0_loss + dur_loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss

from tqdm import tqdm

for epoch in tqdm(range(50), desc="Prosody Training"):
    loss = train_prosody_epoch(prosody_model, prosody_loader, spk_lookup, optim_p)
    tqdm.write(f"Epoch {epoch}  Loss: {loss:.4f}")


torch.save({
    "state_dict": prosody_model.state_dict()
}, "prosody_predictor.pt")


Prosody Training:   2%|▏         | 1/50 [00:14<12:12, 14.94s/it]

Epoch 0  Loss: 10603012.7324


Prosody Training:   4%|▍         | 2/50 [00:29<11:57, 14.95s/it]

Epoch 1  Loss: 9339382.1533


Prosody Training:   6%|▌         | 3/50 [00:44<11:43, 14.96s/it]

...

Epoch 47  Loss: 1579350.1609


Prosody Training:  98%|█████████▊| 49/50 [11:58<00:14, 14.22s/it]

Epoch 48  Loss: 1570430.1652


Prosody Training: 100%|██████████| 50/50 [12:13<00:00, 14.66s/it]

Epoch 49  Loss: 1565814.9006


In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import soundfile as sf
import torchaudio
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

DATA_ROOT = "/data/b22_shruti_chaudhary/.cache/kagglehub/datasets/phantasm34/emovdb-sorted/versions/1"
sample_rate = 16000
hop_length = 256
device = "cuda" if torch.cuda.is_available() else "cpu"

discrete = torch.load("discrete_units.pt", weights_only=False)
paths_kaggle = discrete["paths"]
units_full = discrete["units"]
units_dedup = discrete["units_dedup"]

def to_rel(p):
    return "/".join(p.split("/")[-3:])

rel_from_kaggle = [to_rel(p) for p in paths_kaggle]

df = pd.read_csv("emovdb_manifest.csv")
df["relpath"] = df["filepath"].apply(to_rel)

label_map = torch.load("label_mappings.pt")
emotion2id = label_map["emotion2id"]
id2emotion = {v:k for k,v in emotion2id.items()}
num_emotions = len(emotion2id)

spk_table = torch.load("speaker_table.pt")
speaker2id = spk_table["speaker2id"]
id2speaker = {v:k for k,v in speaker2id.items()}
spk_dim = spk_table["spk_dim"]
num_speakers = len(speaker2id)

prosody_data = torch.load("prosody_data.pt")
prosody_entries = prosody_data["entries"]

rel2prosody = {e["relpath"]: e for e in prosody_entries}

rel2units_dedup = {to_rel(p): u for p,u in zip(paths_kaggle, units_dedup)}

K = 200
PAD = 0
SHIFT = 1
VOCAB_SIZE = K + SHIFT + 2

def get_local_path(rel):
    return os.path.join(DATA_ROOT, rel)

def get_speaker_id(relpath):
    row = df[df["relpath"] == relpath]
    if row.empty:
        return 0
    spk = row.iloc[0]["speaker"]
    return speaker2id.get(spk, 0)

def get_emotion_id(relpath):
    row = df[df["relpath"] == relpath]
    if row.empty:
        return 0
    emo = row.iloc[0]["emotion"]
    return emotion2id.get(emo, 0)

class VocoderDataset(Dataset):
    def __init__(self, entries, max_frames=256):
        self.entries = entries
        self.max_frames = max_frames

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        e = self.entries[idx]
        rel = e["relpath"]
        units_d = e["units_dedup"]
        f0_u = e["f0"]
        dur = e["dur"]
        spk_id = get_speaker_id(rel)
        emo_id = get_emotion_id(rel)

        local_path = get_local_path(rel)
        wav, sr = sf.read(local_path)
        if wav.ndim > 1:
            wav = wav.mean(axis=1)
        if sr != sample_rate:
            wav_t = torch.from_numpy(wav).float().unsqueeze(0)
            wav_t = torchaudio.functional.resample(wav_t, sr, sample_rate)
            wav = wav_t.squeeze(0).numpy()
            sr = sample_rate

        u = units_d.long()
        f0u = f0_u.float()
        d = dur.long()

        unit_frames = []
        f0_frames = []
        for uid, f0v, L in zip(u.tolist(), f0u.tolist(), d.tolist()):
            if L <= 0:
                continue
            unit_frames.extend([uid] * L)
            f0_frames.extend([f0v] * L)

        unit_frames = torch.tensor(unit_frames, dtype=torch.long)
        f0_frames = torch.tensor(f0_frames, dtype=torch.float32)

        if len(unit_frames) == 0:
            unit_frames = torch.zeros(1, dtype=torch.long)
            f0_frames = torch.zeros(1, dtype=torch.float32)

        T = len(unit_frames)
        max_T = self.max_frames

        if T > max_T:
            start = np.random.randint(0, T - max_T + 1)
            end = start + max_T
            unit_frames = unit_frames[start:end]
            f0_frames = f0_frames[start:end]
            T = max_T
        else:
            pad_T = max_T - T
            unit_frames = torch.cat(
                [unit_frames, torch.full((pad_T,), PAD, dtype=torch.long)]
            )
            f0_frames = torch.cat(
                [f0_frames, torch.zeros(pad_T, dtype=torch.float32)]
            )
            T = max_T

        audio_len = T * hop_length

        if len(wav) < audio_len:
            pad = audio_len - len(wav)
            wav = np.pad(wav, (0, pad))
        else:
            start = np.random.randint(0, len(wav) - audio_len + 1)
            wav = wav[start:start + audio_len]

        wav = torch.tensor(wav, dtype=torch.float32)

        return {
            "units": unit_frames,
            "f0": f0_frames,
            "audio": wav,
            "spk_id": torch.tensor(spk_id, dtype=torch.long),
            "emo_id": torch.tensor(emo_id, dtype=torch.long),
        }


def collate_vocoder(batch):
    units = torch.stack([b["units"] for b in batch])
    f0 = torch.stack([b["f0"] for b in batch])
    audio = torch.stack([b["audio"] for b in batch])
    spk = torch.stack([b["spk_id"] for b in batch])
    emo = torch.stack([b["emo_id"] for b in batch])
    return {
        "units": units,
        "f0": f0,
        "audio": audio,
        "spk_id": spk,
        "emo_id": emo,
    }


# class VocoderDataset(Dataset):
#     def __init__(self, entries, max_frames=800):
#         self.entries = entries
#         self.max_frames = max_frames
#     def __len__(self):
#         return len(self.entries)
#     def __getitem__(self, idx):
#         e = self.entries[idx]
#         rel = e["relpath"]
#         units_d = e["units_dedup"]
#         f0_u = e["f0"]
#         dur = e["dur"]
#         spk_id = get_speaker_id(rel)
#         emo_id = get_emotion_id(rel)
#         local_path = get_local_path(rel)
#         wav, sr = sf.read(local_path)
#         if wav.ndim > 1:
#             wav = wav.mean(axis=1)
#         if sr != sample_rate:
#             wav_t = torch.from_numpy(wav).float().unsqueeze(0)
#             wav_t = torchaudio.functional.resample(wav_t, sr, sample_rate)
#             wav = wav_t.squeeze(0).numpy()
#             sr = sample_rate
#         u = units_d.long()
#         f0u = f0_u.float()
#         d = dur.long()
#         unit_frames = []
#         f0_frames = []
#         for uid, f0v, L in zip(u.tolist(), f0u.tolist(), d.tolist()):
#             if L <= 0:
#                 continue
#             unit_frames.extend([uid] * L)
#             f0_frames.extend([f0v] * L)
#         unit_frames = torch.tensor(unit_frames, dtype=torch.long)
#         f0_frames = torch.tensor(f0_frames, dtype=torch.float32)
#         if len(unit_frames) == 0:
#             unit_frames = torch.zeros(1, dtype=torch.long)
#             f0_frames = torch.zeros(1, dtype=torch.float32)
#         T = len(unit_frames)
#         max_T = min(T, self.max_frames)
#         if T > max_T:
#             start = np.random.randint(0, T - max_T + 1)
#             end = start + max_T
#             unit_frames = unit_frames[start:end]
#             f0_frames = f0_frames[start:end]
#             T = max_T
#         max_audio_len = T * hop_length
#         if len(wav) < max_audio_len:
#             pad = max_audio_len - len(wav)
#             wav = np.pad(wav, (0, pad))
#         else:
#             start = np.random.randint(0, len(wav) - max_audio_len + 1)
#             wav = wav[start:start + max_audio_len]
#         wav = torch.tensor(wav, dtype=torch.float32)
#         return {
#             "units": unit_frames,
#             "f0": f0_frames,
#             "audio": wav,
#             "spk_id": torch.tensor(spk_id, dtype=torch.long),
#             "emo_id": torch.tensor(emo_id, dtype=torch.long),
#         }

# def collate_vocoder(batch):
#     max_T = max(len(b["units"]) for b in batch)
#     units = []
#     f0 = []
#     audio = []
#     spk = []
#     emo = []
#     for b in batch:
#         L = len(b["units"])
#         pad_T = max_T - L
#         units.append(torch.cat([b["units"], torch.full((pad_T,), PAD, dtype=torch.long)]))
#         f0.append(torch.cat([b["f0"], torch.zeros(pad_T)]))
#         audio.append(b["audio"])
#         spk.append(b["spk_id"])
#         emo.append(b["emo_id"])
#     units = torch.stack(units)
#     f0 = torch.stack(f0)
#     audio = torch.stack(audio)
#     spk = torch.stack(spk)
#     emo = torch.stack(emo)
#     return {
#         "units": units,
#         "f0": f0,
#         "audio": audio,
#         "spk_id": spk,
#         "emo_id": emo,
#     }

vocoder_ds = VocoderDataset(prosody_entries)
vocoder_loader = DataLoader(vocoder_ds, batch_size=2, shuffle=True, collate_fn=collate_vocoder)

class SpeakerLookup(nn.Module):
    def __init__(self, num_speakers, spk_dim):
        super().__init__()
        self.embedding = nn.Embedding(num_speakers, spk_dim)
    def forward(self, spk_ids):
        return self.embedding(spk_ids)

spk_lookup = SpeakerLookup(num_speakers, spk_dim)
spk_lookup.load_state_dict(spk_table["state_dict"])
spk_lookup = spk_lookup.to(device)
spk_lookup.eval()

emo_dim = 32
unit_emb_dim = 256

class VocoderConditioner(nn.Module):
    def __init__(self, vocab_size, num_emotions, spk_dim, unit_emb_dim=256, emo_dim=32):
        super().__init__()
        self.unit_emb = nn.Embedding(vocab_size, unit_emb_dim, padding_idx=PAD)
        self.emo_emb = nn.Embedding(num_emotions, emo_dim)
        self.proj_spk = nn.Linear(spk_dim, emo_dim)
    def forward(self, units, f0, spk_vecs, emo_ids):
        u = self.unit_emb(units + SHIFT)
        f0 = f0.unsqueeze(-1)
        emo = self.emo_emb(emo_ids).unsqueeze(1)
        spk_p = self.proj_spk(spk_vecs).unsqueeze(1)
        emo_exp = emo.expand(-1, u.size(1), -1)
        spk_exp = spk_p.expand(-1, u.size(1), -1)
        cond = torch.cat([u, f0, emo_exp, spk_exp], dim=-1)
        cond = cond.transpose(1,2)
        return cond

class ResBlock(nn.Module):
    def __init__(self, channels, kernel_size=3, dilation=(1,3,5)):
        super().__init__()
        self.convs1 = nn.ModuleList([
            nn.utils.weight_norm(nn.Conv1d(channels, channels, kernel_size, 1, padding=d*(kernel_size-1)//2, dilation=d))
            for d in dilation
        ])
        self.convs2 = nn.ModuleList([
            nn.utils.weight_norm(nn.Conv1d(channels, channels, kernel_size, 1, padding=(kernel_size-1)//2))
            for _ in dilation
        ])
    def forward(self, x):
        for c1, c2 in zip(self.convs1, self.convs2):
            y = F.leaky_relu(x, 0.1)
            y = c1(y)
            y = F.leaky_relu(y,0.1)
            y = c2(y)
            x = x + y
        return x

class HiFiGAN_Generator(nn.Module):
    def __init__(self, cond_dim, channels=512, upsample_rates=(8,8,2,2), upsample_kernel_sizes=(16,16,4,4)):
        super().__init__()
        self.conv_pre = nn.utils.weight_norm(nn.Conv1d(cond_dim, channels,7,1,padding=3))
        self.ups = nn.ModuleList()
        self.resblocks = nn.ModuleList()
        in_ch = channels
        for u,k in zip(upsample_rates, upsample_kernel_sizes):
            self.ups.append(
                nn.utils.weight_norm(
                    nn.ConvTranspose1d(in_ch,in_ch//2,k,u,padding=(k-u)//2)
                )
            )
            in_ch = in_ch//2
            self.resblocks.append(nn.ModuleList([ResBlock(in_ch) for _ in range(3)]))
        self.conv_post = nn.utils.weight_norm(nn.Conv1d(in_ch,1,7,1,padding=3))
    def forward(self,x):
        x = self.conv_pre(x)
        for up,rbs in zip(self.ups,self.resblocks):
            x = F.leaky_relu(x,0.1)
            x = up(x)
            out = 0
            for rb in rbs:
                out += rb(x)
            x = out/len(rbs)
        x = F.leaky_relu(x,0.1)
        x = self.conv_post(x)
        x = torch.tanh(x)
        return x

class MPD_Block(nn.Module):
    def __init__(self, period):
        super().__init__()
        self.period = period
        self.convs = nn.ModuleList([
            nn.utils.weight_norm(nn.Conv2d(1,32,(5,1),(3,1),padding=(2,0))),
            nn.utils.weight_norm(nn.Conv2d(32,128,(5,1),(3,1),padding=(2,0))),
            nn.utils.weight_norm(nn.Conv2d(128,512,(5,1),(3,1),padding=(2,0))),
            nn.utils.weight_norm(nn.Conv2d(512,1024,(5,1),(3,1),padding=(2,0))),
            nn.utils.weight_norm(nn.Conv2d(1024,1024,(5,1),1,padding=(2,0)))
        ])
        self.conv_post = nn.utils.weight_norm(nn.Conv2d(1024,1,(3,1),1,padding=(1,0)))
    def forward(self,x):
        b,c,t = x.shape
        # if t % self.period != 0:
        #     pad = self.period - (t%self.period)
        #     x = F.pad(x,(0,pad),"reflect")
        pad = (self.period - (t % self.period)) % self.period
        x = F.pad(x,(0,pad),mode="reflect")
        t = x.shape[-1]
        x = x.view(b,1,t//self.period,self.period)
        feats = []
        for l in self.convs:
            x = l(x)
            x = F.leaky_relu(x,0.1)
            feats.append(x)
        x = self.conv_post(x)
        feats.append(x)
        x = x.reshape(b,-1)
        return x,feats

class MultiPeriodDiscriminator(nn.Module):
    def __init__(self, periods=[2,3,5,7,11]):
        super().__init__()
        self.blocks = nn.ModuleList([MPD_Block(p) for p in periods])
    def forward(self,x):
        ret=[]
        for b in self.blocks:
            score,feat = b(x)
            ret.append((score,feat))
        return ret

class MSD_Block(nn.Module):
    def __init__(self):
        super().__init__()
        self.convs = nn.ModuleList([
            nn.utils.weight_norm(nn.Conv1d(1,128,15,1,padding=7)),
            nn.utils.weight_norm(nn.Conv1d(128,128,41,2,padding=20,groups=4)),
            nn.utils.weight_norm(nn.Conv1d(128,256,41,2,padding=20,groups=16)),
            nn.utils.weight_norm(nn.Conv1d(256,512,41,2,padding=20,groups=16)),
            nn.utils.weight_norm(nn.Conv1d(512,512,41,2,padding=20,groups=16)),
            nn.utils.weight_norm(nn.Conv1d(512,512,5,1,padding=2)),
        ])
        self.conv_post = nn.utils.weight_norm(nn.Conv1d(512,1,3,1,padding=1))
    def forward(self,x):
        feats=[]
        for l in self.convs:
            x=l(x)
            x=F.leaky_relu(x,0.1)
            feats.append(x)
        x=self.conv_post(x)
        feats.append(x)
        return x,feats

class MultiScaleDiscriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.b0 = MSD_Block()
        self.b1 = MSD_Block()
        self.b2 = MSD_Block()
        self.avg = nn.AvgPool1d(4,2,padding=2)
    def forward(self,x):
        outs=[]
        x1=x
        outs.append(self.b0(x1))
        x2=self.avg(x1)
        outs.append(self.b1(x2))
        x3=self.avg(x2)
        outs.append(self.b2(x3))
        return outs

def loss_adv_g(scores):
    loss=0
    for s in scores:
        loss+=torch.mean((1-s)**2)
    return loss

def loss_adv_d(real_scores,fake_scores):
    loss=0
    for r,f in zip(real_scores,fake_scores):
        loss+=torch.mean((r-1)**2)+torch.mean(f**2)
    return loss

def feature_matching(real_feats,fake_feats):
    loss=0
    for r,f in zip(real_feats,fake_feats):
        for rr,ff in zip(r,f):
            loss+=torch.mean(torch.abs(rr-ff))
    return loss

def log_norm_f0(f0):
    f0 = f0.clone()
    f0[f0 <= 0] = 1.0
    f0 = torch.log(f0)
    m = f0.mean()
    s = f0.std().clamp(min=1e-6)
    return (f0 - m) / s

cond_dim = unit_emb_dim + 1 + emo_dim + emo_dim

generator = HiFiGAN_Generator(cond_dim=cond_dim).to(device)
mpd = MultiPeriodDiscriminator().to(device)
msd = MultiScaleDiscriminator().to(device)
cond_net = VocoderConditioner(VOCAB_SIZE, num_emotions, spk_dim, unit_emb_dim=unit_emb_dim, emo_dim=emo_dim).to(device)

optim_g = torch.optim.Adam(generator.parameters(), lr=2e-4, betas=(0.8,0.99))
optim_d = torch.optim.Adam(list(mpd.parameters())+list(msd.parameters()), lr=2e-4, betas=(0.8,0.99))

def discriminator_step(batch):
    units = batch["units"].to(device)
    f0 = batch["f0"].to(device)
    audio = batch["audio"].to(device)
    spk_id = batch["spk_id"].to(device)
    emo_id = batch["emo_id"].to(device)
    with torch.no_grad():
        spk_vecs = spk_lookup(spk_id)
        f0n = log_norm_f0(f0)
        cond = cond_net(units, f0n, spk_vecs, emo_id)
        fake_audio = generator(cond).squeeze(1)
        fake_audio = fake_audio[:, :audio.size(1)]
    real = audio.unsqueeze(1)
    fake = fake_audio.unsqueeze(1)
    r_mpd = mpd(real)
    f_mpd = mpd(fake)
    r_msd = msd(real)
    f_msd = msd(fake)
    real_scores = [x[0] for x in r_mpd] + [x[0] for x in r_msd]
    fake_scores = [x[0] for x in f_mpd] + [x[0] for x in f_msd]
    loss_d = loss_adv_d(real_scores, fake_scores)
    return loss_d

def generator_step(batch):
    units = batch["units"].to(device)
    f0 = batch["f0"].to(device)
    audio = batch["audio"].to(device)
    spk_id = batch["spk_id"].to(device)
    emo_id = batch["emo_id"].to(device)
    spk_vecs = spk_lookup(spk_id)
    f0n = log_norm_f0(f0)
    cond = cond_net(units, f0n, spk_vecs, emo_id)
    fake_audio = generator(cond).squeeze(1)
    fake_audio = fake_audio[:, :audio.size(1)]
    real = audio.unsqueeze(1)
    fake = fake_audio.unsqueeze(1)
    r_mpd = mpd(real)
    f_mpd = mpd(fake)
    r_msd = msd(real)
    f_msd = msd(fake)
    adv = loss_adv_g([x[0] for x in f_mpd] + [x[0] for x in f_msd])
    fm = feature_matching([x[1] for x in r_mpd]+[x[1] for x in r_msd],[x[1] for x in f_mpd]+[x[1] for x in f_msd])
    l1 = F.l1_loss(fake_audio,audio)
    loss_g = adv + 10*l1 + 2*fm
    return loss_g

num_epochs = 50  # set >0 when you want to train

for epoch in range(num_epochs):

    generator.train()
    mpd.train()
    msd.train()

    total_g = 0.0
    total_d = 0.0

    # progress bar over batches
    pbar = tqdm(vocoder_loader, desc=f"Epoch {epoch+1}/{num_epochs}", ncols=120)

    for batch in pbar:

        # ---- discriminator ----
        optim_d.zero_grad()
        ld = discriminator_step(batch)
        ld.backward()
        optim_d.step()

        # ---- generator ----
        optim_g.zero_grad()
        lg = generator_step(batch)
        lg.backward()
        optim_g.step()

        total_d += ld.item()
        total_g += lg.item()

        # update display
        pbar.set_postfix({
            "D_loss": f"{ld.item():.4f}",
            "G_loss": f"{lg.item():.4f}",
            "D_epoch": f"{total_d:.2f}",
            "G_epoch": f"{total_g:.2f}",
        })

    print(f"\nEpoch {epoch+1}/{num_epochs} DONE | D={total_d:.2f} | G={total_g:.2f}\n")

    # save every 50 epochs
    if (epoch + 1) % 10 == 0:
        ckpt_path = f"hifigan_vocoder_epoch{epoch+1}.pt"
        torch.save({
            "generator": generator.state_dict(),
            "mpd": mpd.state_dict(),
            "msd": msd.state_dict(),
            "cond_net": cond_net.state_dict()
        }, ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)
    def forward(self, x):
        return self.pe[:, :x.size(1)].to(x.device)

class EmotionTransformer(nn.Module):
    def __init__(self, num_emotions, vocab_size=VOCAB_SIZE, d_model=512, nhead=8, num_layers=6, dim_feedforward=2048, dropout=0.1, max_len=4096):
        super().__init__()
        self.num_emotions = num_emotions
        self.vocab_size = vocab_size
        self.token_emb = nn.Embedding(vocab_size, d_model, padding_idx=PAD)
        self.pos_enc = PositionalEncoding(d_model, max_len=max_len)
        self.encoders = nn.ModuleDict()
        self.decoders = nn.ModuleDict()
        enc_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        dec_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        for e in range(num_emotions):
            self.encoders[str(e)] = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
            self.decoders[str(e)] = nn.TransformerDecoder(dec_layer, num_layers=num_layers)
        self.output_proj = nn.Linear(d_model, vocab_size)
        self.d_model = d_model
    def encode(self, src_tokens, src_mask, emotion_id):
        e = str(int(emotion_id))
        x = self.token_emb(src_tokens) * np.sqrt(self.d_model)
        x = x + self.pos_enc(x)
        x = x.transpose(0,1)
        memory = self.encoders[e](x, src_key_padding_mask=src_mask)
        return memory
    def decode(self, tgt_tokens, memory, memory_mask, emotion_id, tgt_mask):
        e = str(int(emotion_id))
        y = self.token_emb(tgt_tokens) * np.sqrt(self.d_model)
        y = y + self.pos_enc(y)
        y = y.transpose(0,1)
        out = self.decoders[e](y, memory, tgt_mask=tgt_mask, memory_key_padding_mask=memory_mask)
        out = out.transpose(0,1)
        logits = self.output_proj(out)
        return logits
    def generate_square_subsequent_mask(self, sz):
        return torch.triu(torch.full((sz, sz), float('-inf')), diagonal=1)

def greedy_decode_es2s(model, src, src_mask, src_em, tgt_em, max_len=400):
    model.eval()
    with torch.no_grad():
        memory = model.encode(src, src_mask, src_em)
        ys = torch.tensor([[VOCAB_SIZE-2]], dtype=torch.long).to(device)
        for _ in range(max_len):
            tgt_mask = model.generate_square_subsequent_mask(ys.size(1)).to(device)
            out = model.decode(ys, memory, src_mask, tgt_em, tgt_mask)
            next_token = out[:, -1, :].argmax(-1, keepdim=True)
            ys = torch.cat([ys, next_token], dim=1)
            if next_token.item() == VOCAB_SIZE-1:
                break
    return ys.squeeze(0).tolist()

class ProsodyPredictor(nn.Module):
    def __init__(self, vocab_size, num_emotions, spk_dim, d_model=256, nhead=4, num_layers=3, ff=512):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, d_model, padding_idx=PAD)
        self.emo_emb = nn.Embedding(num_emotions, d_model)
        self.proj_spk = nn.Linear(spk_dim, d_model)
        enc_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=ff, batch_first=True)
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
        self.f0_head = nn.Linear(d_model, 1)
        self.dur_head = nn.Linear(d_model, 1)
    def forward(self, units, emo_ids, spk_vecs, src_key_padding_mask):
        x_tok = self.token_emb(units)
        x_emo = self.emo_emb(emo_ids).unsqueeze(1)
        x_spk = self.proj_spk(spk_vecs).unsqueeze(1)
        x = x_tok + x_emo + x_spk
        h = self.encoder(x, src_key_padding_mask=src_key_padding_mask)
        f0 = self.f0_head(h).squeeze(-1)
        dur = self.dur_head(h).squeeze(-1)
        return f0, dur

es2s = EmotionTransformer(num_emotions=num_emotions).to(device)
es2s.load_state_dict(torch.load("es2s_transformer.pt", map_location=device))
es2s.eval()

prosody_model = ProsodyPredictor(VOCAB_SIZE, num_emotions, spk_dim).to(device)
prosody_model.load_state_dict(torch.load("prosody_predictor.pt", map_location=device)["state_dict"])
prosody_model.eval()

def expand_units_with_dur(units, dur):
    units = units.tolist()
    dur = torch.clamp(dur, min=1.0).round().long().tolist()
    u_frames = []
    for u,d in zip(units,dur):
        u_frames.extend([u]*d)
    return torch.tensor(u_frames, dtype=torch.long)

def convert_emotion_from_relpath(relpath, tgt_emotion_name, vocoder_ckpt, max_len_units=400):
    tgt_emo_id = emotion2id[tgt_emotion_name]
    src_emo_id = get_emotion_id(relpath)
    spk_id = get_speaker_id(relpath)
    spk_vec = spk_lookup(torch.tensor([spk_id],dtype=torch.long).to(device))
    units_src = rel2units_dedup[relpath]
    src = torch.tensor([[u+SHIFT for u in units_src]], dtype=torch.long).to(device)
    src_mask = torch.zeros_like(src, dtype=torch.bool).to(device)
    src_em = torch.tensor(src_emo_id, dtype=torch.long).to(device)
    tgt_em = torch.tensor(tgt_emo_id, dtype=torch.long).to(device)
    out_units = greedy_decode_es2s(es2s, src, src_mask, src_em, tgt_em, max_len=max_len_units)
    out_units = [u for u in out_units if u > SHIFT and u < VOCAB_SIZE-1]
    units_tgt = torch.tensor(out_units, dtype=torch.long).unsqueeze(0).to(device)
    mask = torch.zeros_like(units_tgt, dtype=torch.bool).to(device)
    emo_ids = torch.tensor([tgt_emo_id], dtype=torch.long).to(device)
    f0_pred, dur_pred = prosody_model(units_tgt, emo_ids, spk_vec, mask)
    units_frames = expand_units_with_dur(units_tgt[0].cpu(), dur_pred[0].cpu())
    f0_frames = torch.zeros_like(units_frames, dtype=torch.float32)
    cond_len = len(units_frames)
    ckpt = torch.load(vocoder_ckpt, map_location=device)
    generator.load_state_dict(ckpt["generator"])
    cond_net.load_state_dict(ckpt["cond_net"])
    generator.eval()
    cond_units = units_frames.unsqueeze(0).to(device)
    cond_f0 = f0_frames.unsqueeze(0).to(device)
    emo_ids_b = emo_ids
    spk_vecs_b = spk_vec
    f0n = log_norm_f0(cond_f0)
    cond = cond_net(cond_units, f0n, spk_vecs_b, emo_ids_b)
    with torch.no_grad():
        audio_hat = generator(cond).squeeze(1)[0].cpu().numpy()
    return audio_hat



/data/b22_shruti_chaudhary/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Epoch 1/50:  89%|▉| 3062/3447 [20:51<02:37,  2.44it/s, D_loss=3.2088, G_loss=10.3493, D_epoch=10600.84, G_epoch=25093.76

In [1]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print("  GPU", i, ":", torch.cuda.get_device_name(i))


CUDA available: True
Number of GPUs: 3
  GPU 0 : NVIDIA RTX A6000
  GPU 1 : NVIDIA RTX A5000
  GPU 2 : NVIDIA RTX A5000


In [5]:
import os
import torch
import soundfile as sf
import torchaudio
import numpy as np
import torchcrepe
from tqdm import tqdm

# Load prosody entries (fix NameError)
prosody_data = torch.load("prosody_data.pt", map_location="cpu")
prosody_entries = prosody_data["entries"]

DATA_ROOT = "/data/b22_shruti_chaudhary/.cache/kagglehub/datasets/phantasm34/emovdb-sorted/versions/1"
SAVE_DIR = "f0_cache/"
os.makedirs(SAVE_DIR, exist_ok=True)

SAMPLE_RATE = 16000
HOP = 256

def get_f0(wav, sr, hop=HOP):
    # Resample to 16k if needed
    if sr != SAMPLE_RATE:
        wav = torchaudio.functional.resample(
            torch.tensor(wav).unsqueeze(0),
            sr, SAMPLE_RATE
        ).squeeze(0).numpy()
        sr = SAMPLE_RATE

    # Convert to GPU
    audio = torch.tensor(wav, dtype=torch.float32).unsqueeze(0).cuda()
   # print(audio.device)


    # Run torchcrepe
    with torch.no_grad():
        f0 = torchcrepe.predict(audio, sr, hop, 50.0, 500.0)
        f0 = f0.squeeze(0).cpu().numpy()

    return f0

for entry in tqdm(prosody_entries):
    rel = entry["relpath"]
    wav_path = os.path.join(DATA_ROOT, rel)

    wav, sr = sf.read(wav_path)
    
    # Convert to mono
    if wav.ndim > 1:
        wav = wav.mean(axis=1)

    f0 = get_f0(wav, sr)

    save_name = rel.replace("/", "_") + ".pt"
    save_path = os.path.join(SAVE_DIR, save_name)

    torch.save(torch.tensor(f0), save_path)

print("F0 extraction complete!")


100%|██████████| 6893/6893 [5:31:55<00:00,  2.89s/it]  

F0 extraction complete!
